# PaniniFS Research - Traitement Corpus Multilingue\n\nCollecte et analyse de corpus cross-linguistique avec GPU\n\n## 🌐 Configuration Multilingue

In [ ]:
# Installation des dépendances multilingues\n!pip install transformers torch\n!pip install spacy polyglot langdetect\n!pip install requests beautifulsoup4 feedparser\n!pip install wikipedia arxiv\n\n# Modèles spaCy multilingues\n!python -m spacy download en_core_web_sm\n!python -m spacy download fr_core_news_sm\n!python -m spacy download de_core_news_sm\n\nimport requests\nimport json\nimport wikipedia\nimport feedparser\nfrom langdetect import detect\nimport spacy\nfrom transformers import pipeline

In [ ]:
class ColabCorpusCollector:\n    def __init__(self):\n        self.languages = ['en', 'fr', 'de', 'es', 'it']\n        self.sources = {\n            'wikipedia': self.collect_wikipedia,\n            'arxiv': self.collect_arxiv,\n            'news_feeds': self.collect_news_feeds\n        }\n        \n    def collect_wikipedia(self, topics, max_articles=50):\n        articles = []\n        \n        for lang in self.languages:\n            wikipedia.set_lang(lang)\n            \n            for topic in topics:\n                try:\n                    search_results = wikipedia.search(topic, results=10)\n                    \n                    for title in search_results[:5]:\n                        try:\n                            page = wikipedia.page(title)\n                            articles.append({\n                                'source': 'wikipedia',\n                                'language': lang,\n                                'topic': topic,\n                                'title': page.title,\n                                'content': page.content[:2000],\n                                'url': page.url\n                            })\n                            \n                            if len(articles) >= max_articles:\n                                return articles\n                                \n                        except Exception as e:\n                            continue\n                            \n                except Exception as e:\n                    continue\n                    \n        return articles\n        \n    def collect_arxiv(self, topics, max_papers=30):\n        papers = []\n        \n        for topic in topics:\n            search_url = f'http://export.arxiv.org/api/query?search_query=all:{topic}&max_results=10'\n            \n            try:\n                response = requests.get(search_url)\n                \n                if response.status_code == 200:\n                    import xml.etree.ElementTree as ET\n                    root = ET.fromstring(response.content)\n                    \n                    for entry in root.findall('.//{http://www.w3.org/2005/Atom}entry'):\n                        title_elem = entry.find('.//{http://www.w3.org/2005/Atom}title')\n                        summary_elem = entry.find('.//{http://www.w3.org/2005/Atom}summary')\n                        \n                        if title_elem is not None and summary_elem is not None:\n                            papers.append({\n                                'source': 'arxiv',\n                                'topic': topic,\n                                'title': title_elem.text,\n                                'content': summary_elem.text[:1500],\n                                'language': 'en'\n                            })\n                            \n                        if len(papers) >= max_papers:\n                            return papers\n                            \n            except Exception as e:\n                continue\n                \n        return papers\n        \n    def collect_multilingual_corpus(self, topics):\n        all_documents = []\n        \n        print('🌐 Collecte Wikipedia multilingue...')\n        wikipedia_docs = self.collect_wikipedia(topics)\n        all_documents.extend(wikipedia_docs)\n        print(f'✅ {len(wikipedia_docs)} articles Wikipedia collectés')\n        \n        print('📚 Collecte ArXiv...')\n        arxiv_docs = self.collect_arxiv(topics)\n        all_documents.extend(arxiv_docs)\n        print(f'✅ {len(arxiv_docs)} papers ArXiv collectés')\n        \n        return all_documents\n\n# Initialisation\ncollector = ColabCorpusCollector()\n\n# Topics de recherche linguistique\nlinguistic_topics = [\n    'morphology', 'syntax', 'semantics', 'phonology',\n    'computational linguistics', 'natural language processing',\n    'grammar theory', 'linguistic typology'\n]\n\n# Collecte\ncorpus_documents = collector.collect_multilingual_corpus(linguistic_topics)\nprint(f'\\n🎯 Total corpus: {len(corpus_documents)} documents')